###  9. Языковое моделирование

#### Задание<br>
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор


Подготовим окружение

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
import time

Посмотрим на данные

In [4]:
from google.colab import files

In [5]:
uploaded = files.upload()

Saving evgenyi_onegin.txt to evgenyi_onegin.txt


In [6]:
path = 'evgenyi_onegin.txt'

In [7]:
#удалим спецсимволы
text = open(path, 'rb').read().decode(encoding='utf-8')
text = text.replace('\ufeff', '')
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [8]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [9]:
text = text + text

In [10]:
# Уникальные символы в файле
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [11]:
import numpy as np
# сопоставим уникальныуе символы с индексами
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [12]:
text_as_int, text[:500], len(text_as_int), len(text) #символьная и числовая длина совпадает

(array([ 71, 110, 104, ..., 104, 121,   0]),
 'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                                Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Святой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высо',
 573968,
 573968)

In [13]:
#максимальная длина предложения
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

#создадим тренировочный датасет примеров
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [14]:
#размер последовательностей возьмем по 5
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [15]:
#создадим целевую переменную в числовом и символьном представлении
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [16]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


Создадим кастомную модель на основе LSTM слоев

In [17]:
# размер партии входящих символов
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [18]:
vocab_size = len(vocab) # длина символьного словаря

embedding_dim = 256     #размеры эмбедингов

rnn_units = 1024        #количество единиц RNN

In [19]:
def custom_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [20]:
model = custom_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions, "# (batch_size, sequence_length, vocab_size)")

tf.Tensor(
[[[ 9.82167876e-08 -1.31884935e-05  7.87777753e-06 ...  7.91770799e-06
    1.46531102e-05  3.79539938e-06]
  [-3.74233878e-06 -3.29119393e-05  2.52597147e-05 ...  2.00612685e-05
    3.29373797e-05 -3.32847685e-06]
  [-6.95966310e-06 -5.10529208e-05  6.47224952e-05 ...  3.85709791e-05
    4.85859928e-05 -3.31991650e-05]
  ...
  [-2.26237928e-03 -3.90460691e-03  5.52886678e-03 ...  2.68567004e-04
    7.69921998e-03 -1.67382369e-03]
  [-2.36142986e-03 -4.03757952e-03  5.07123116e-03 ...  3.94784962e-04
    7.61754485e-03 -1.66451256e-03]
  [-2.40765372e-03 -4.15153662e-03  4.54044528e-03 ...  5.24297706e-04
    7.49317836e-03 -1.64235989e-03]]

 [[ 1.01610349e-05 -1.10081119e-05 -2.56513704e-05 ...  1.38181749e-05
    2.69811608e-05  5.10676728e-06]
  [ 3.53446849e-05 -3.89087545e-05 -5.55728111e-05 ...  2.86995237e-05
    8.35690298e-05  1.31986471e-05]
  [ 7.84848744e-05 -9.46172513e-05 -5.94844496e-05 ...  2.63692236e-05
    1.53860456e-04  1.40801194e-05]
  ...
  [-4.795412

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30592899 (116.70 MB)
Trainable params: 305

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Ввод: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Прогнозируемое продолжение текста: \n", repr("".join(idx2char[sampled_indices ])))

Ввод: 
 'ет и шумит...\n                        "Погибну, - Таня говорит, -\n                        Но гибель '

Прогнозируемое продолжение текста: 
 'У2Y?lОвОMуъосuФ.)3D цIпАяG9ЧXдыGрOэ\nsSEЭoН1ыlуR01чх(ЦFОM1ЕМtхОддЭ1AmQщИмЛБQЭъTXПLДплЯяIСонzэтгМ}п-4l'


Как видно, с прогнозом все очень плохо

In [24]:
#добавим функцию потерь
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.8758535


In [25]:
#скомпилируем модель
model.compile(optimizer='adam', loss=loss)

In [26]:
#создадим папку, в которой будут сохранены контрольные точки
!rm -rf /training_checkpoints

In [27]:
!ls /training_checkpoints

ls: cannot access '/training_checkpoints': No such file or directory


In [28]:
name1 = 'training_checkpoints'

os.mkdir('/{}'.format(name1))

In [29]:
checkpoint_dir = '/training_checkpoints'

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=7*10,
    save_weights_only=True)

Обучение модели

In [30]:
%time
EPOCHS = 25
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 9.06 µs
Epoch 1/25
88/88 [==============================] - 39s 362ms/step - loss: 2.3546
Epoch 2/25
88/88 [==============================] - 33s 369ms/step - loss: 1.8593
Epoch 3/25
88/88 [==============================] - 35s 381ms/step - loss: 1.6293
Epoch 4/25
88/88 [==============================] - 35s 386ms/step - loss: 1.4845
Epoch 5/25
88/88 [==============================] - 34s 379ms/step - loss: 1.4031
Epoch 6/25
88/88 [==============================] - 35s 382ms/step - loss: 1.3679
Epoch 7/25
88/88 [==============================] - 35s 379ms/step - loss: 1.3401
Epoch 8/25
88/88 [==============================] - 35s 392ms/step - loss: 1.3268
Epoch 9/25
88/88 [==============================] - 34s 379ms/step - loss: 1.3044
Epoch 10/25
88/88 [==============================] - 34s 381ms/step - loss: 1.2734
Epoch 11/25
88/88 [==============================] - 34s 379ms/step - loss: 1.2538
Epoch 12/25
88/88 [=============

In [31]:
tf.train.latest_checkpoint(checkpoint_dir)

'/training_checkpoints/ckpt_25'

In [32]:
model = custom_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [33]:
#функция генареции текста

def generate_text(model, start_string, temperature=1, num_generate=500):# Шаг оценки (генерация текста с использованием обученной модели)

    num_generate = num_generate # Количество символов для генерации

    input_eval = [char2idx[s] for s in start_string] # Преобразование нашей стартовой строки в числа (векторизация)
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = [] # Пустая строка для хранения результатов

    temperature = temperature

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature # использование категориального распределения для прогнозирования символа, возвращаемого моделью
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

Посмотрим на резелтат обучения сети

In [34]:
text_ = generate_text(model, start_string=u"Мой дядя самых честных правил ", temperature=1.3)
print(text_)

Мой дядя самых честных правил и замесак.

                              Трещев, и бедною Ленский,
                     XXIV

                        - Сорхчаеные: Пыть по нуе
      vбракго. Ада
     Свено без трузы удругся,
        Сомрейниц урених сельде.

                                   XII

         Россижа, друще   Чуть засета,
      Томевиет, фозятелый,
                     Поскаженным,
                    голды Людувеголья
             Подру     Тум нас?
.. спатнепь, стмой природа,
             VII

 ни, где часпищо


Генерация происходит в случайном порядке

In [35]:
#снизим температуру
text_ = generate_text(model, start_string=u"Мой дядя самых честных правил ", temperature=1.1)
print(text_)

Мой дядя самых честных правил небщо Он? Лес, и взор вознада. Мте в душа:
                         Татьяна жизни за мою
                        Как не врыжателия вин,
                        Все вов верительно врешет
                      - В летгихму сдол к молвало
                        Селится он уго потолым;
                        Круда            И Тан: хоть Вы зномоне слега
                        В сожно    Забось, трепетник и веластель
                        Кпершило все злово б горог
           Кешко бе-V
        


В этом тексте угадываются несколько реальных отрывков, соединенных неудачной генерацией.

In [39]:
#снизим температуру до 0.9. Модель покажет одинаковую генерацию большого длинного отрывка
text_ = generate_text(model, start_string=u"Мой дядя самых честных правил ", temperature=0.9)
print(text_)

Мой дядя самых честных правил не могной
                            Разжеслось они сеобраты;
                        Он с ворожет? Томя звазьной
                        И вдруг молодной обро                      Beosieok                        Прошал перекли не постелся? -
                        И вашествих он отлабилась,
                        Я упозерий пылью.
                                           XIV

                        Трову то жальный и простой
                        Тапыя другает хоть дни болете,
         


In [40]:
text_ = generate_text(model, start_string=u"Мой дядя самых честных правил ", temperature=0.09)
print(text_)

Мой дядя самых честных правил своей
                        И все не волос в садом строгих странной поленой
                        И все не волос на столом
                        И вот не забыть не постели
                        И все под ним под ней подруга
                        И вот не волос и постеле
                        И все в толко слезы подруга
                        Он под ней не после все в ней привета
                        И все не волненье подавить
                        И вот не слышит и полно
      


In [41]:
len(text_)

530

#### Выводы:

Модель воспроизводит длинную последовательность. Вместо составления смешных предложений из отдельных слов воспроизводит один или несколько целых отрывков. Это свидтельствует о признаках переобучения. Для более качественного обучения модели требуется больше текста, чтобы она не запоминала привязки отдельных символов к кускам текста. Различные начальные участки текста генерируют разный начальный символ, что приводит к воспроизведению разных участков текста. Если генерируется по одному символу, то предсказание следующего символа сильно осложнено, если применить к мену целую строку, то генерация приобретает большую степень осмысленности.